# Inteligência Computacional

## Projeto DATA MINING CUP Competition 2013

In [144]:
import pandas as pd

In [145]:
data = pd.read_csv('transact_train.csv')

In [146]:
group = data["sessionNo"]
agg = data.groupby([group])
splited = []
for group in agg:
    splited.append(pd.DataFrame(data = group[1], columns=data.columns.values))

In [147]:
splited[2] # vetor de dataframes separados por sessionNo

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
8,3,6,5,181477.0,9,29.99,29.99,89.97,1,29.99,...,?,3,1800,475,302,12,45,1,11,y
9,3,6,5,297018.0,11,9.99,29.99,109.95,2,9.99,...,?,3,1800,475,302,12,45,1,11,y
10,3,6,5,310967.0,11,9.99,29.99,109.95,2,9.99,...,completely orderable,3,1800,475,302,12,45,1,11,y
11,3,6,5,324278.0,11,9.99,29.99,109.95,2,9.99,...,completely orderable,3,1800,475,302,12,45,1,11,y
12,3,6,5,341613.0,11,9.99,29.99,109.95,2,9.99,...,completely orderable,3,1800,475,302,12,45,1,11,y


In [148]:
bStep_count = []
countLog = []
modes = []
for session in splited:
    countStep = 0
    countLog.append(len(session))
    modes.append(session['bStep'].mode()[0])
    for j in range(len(session)):
        if(session['bStep'].iloc[j] != "?" and int(session['bStep'].iloc[j]) >= 3):
            countStep +=1
    bStep_count.append(countStep)

In [149]:
bStep_count # quantidade de bStep maiores que 3 de cada sessão

[0,
 2,
 1,
 0,
 2,
 0,
 2,
 0,
 3,
 0,
 10,
 9,
 3,
 0,
 0,
 5,
 5,
 0,
 0,
 5,
 3,
 0,
 0,
 1,
 0,
 2,
 0,
 3,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 5,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 2,
 10,
 0,
 0,
 4,
 4,
 0,
 3,
 4,
 1,
 0,
 4,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 13,
 3,
 6,
 0,
 0,
 5,
 4,
 3,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 7,
 0,
 0,
 2,
 1,
 0,
 2,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 5,
 3,
 0,
 0,
 15,
 2,
 0,
 0,
 5,
 0,
 0,
 19,
 0,
 0,
 0,
 1,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 4,
 3,
 2,
 0,
 10,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 1,
 1,
 5,
 0,
 0,
 0,
 0,
 2,
 1,
 7,
 0,
 0,
 1,
 0,
 0,
 0,
 5,
 3,
 3,
 0,
 0,
 3,
 3,
 5,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 3,
 0,
 0,
 3,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 0,
 0,
 3,
 0,
 0,
 0,
 3,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 9,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 1,
 6,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0

In [150]:
countLog #Numero de log de cada sessão

[3,
 5,
 5,
 2,
 8,
 3,
 15,
 1,
 6,
 2,
 20,
 27,
 11,
 4,
 2,
 21,
 6,
 5,
 19,
 8,
 6,
 18,
 2,
 17,
 3,
 33,
 5,
 9,
 13,
 4,
 1,
 13,
 4,
 2,
 1,
 7,
 20,
 2,
 1,
 9,
 2,
 8,
 3,
 2,
 2,
 9,
 2,
 1,
 2,
 6,
 3,
 7,
 5,
 1,
 3,
 1,
 9,
 15,
 3,
 7,
 6,
 8,
 20,
 4,
 15,
 14,
 2,
 3,
 8,
 4,
 3,
 6,
 13,
 3,
 2,
 8,
 1,
 4,
 13,
 5,
 3,
 2,
 2,
 1,
 3,
 6,
 5,
 4,
 27,
 13,
 8,
 14,
 3,
 11,
 6,
 9,
 14,
 1,
 4,
 6,
 6,
 3,
 2,
 1,
 3,
 2,
 15,
 2,
 1,
 5,
 2,
 3,
 2,
 2,
 9,
 2,
 1,
 2,
 8,
 8,
 2,
 2,
 12,
 9,
 15,
 2,
 5,
 11,
 2,
 8,
 2,
 8,
 2,
 15,
 6,
 18,
 1,
 18,
 16,
 3,
 12,
 12,
 2,
 3,
 36,
 2,
 1,
 1,
 22,
 4,
 6,
 2,
 1,
 2,
 2,
 5,
 12,
 7,
 6,
 1,
 25,
 2,
 7,
 1,
 2,
 9,
 1,
 2,
 2,
 8,
 13,
 6,
 7,
 3,
 2,
 11,
 1,
 3,
 38,
 14,
 2,
 1,
 5,
 1,
 2,
 1,
 9,
 9,
 16,
 3,
 1,
 22,
 10,
 12,
 3,
 2,
 15,
 14,
 1,
 4,
 6,
 4,
 18,
 2,
 4,
 5,
 12,
 2,
 2,
 4,
 1,
 6,
 7,
 13,
 2,
 3,
 11,
 2,
 2,
 2,
 6,
 2,
 17,
 8,
 2,
 2,
 6,
 1,
 2,
 6,
 13,
 26,
 5,
 2,
 2,
 1,
 8

In [151]:
modes # moda do bStep de cada Sessão.

['?',
 '4',
 '?',
 '1',
 '?',
 '?',
 '?',
 '?',
 '2',
 '?',
 '?',
 '?',
 '2',
 '1',
 '2',
 '1',
 '3',
 '?',
 '?',
 '4',
 '1',
 '?',
 '2',
 '?',
 '2',
 '1',
 '?',
 '1',
 '2',
 '?',
 '?',
 '2',
 '?',
 '2',
 '?',
 '?',
 '?',
 '2',
 '?',
 '1',
 '1',
 '?',
 '?',
 '?',
 '2',
 '?',
 '1',
 '?',
 '2',
 '?',
 '?',
 '4',
 '?',
 '1',
 '?',
 '?',
 '2',
 '1',
 '?',
 '?',
 '?',
 '?',
 '?',
 '1',
 '?',
 '3',
 '?',
 '?',
 '3',
 '4',
 '?',
 '1',
 '?',
 '?',
 '2',
 '?',
 '?',
 '?',
 '2',
 '?',
 '?',
 '?',
 '2',
 '?',
 '1',
 '2',
 '?',
 '?',
 '1',
 '?',
 '4',
 '?',
 '?',
 '1',
 '4',
 '1',
 '2',
 '?',
 '1',
 '1',
 '2',
 '?',
 '?',
 '?',
 '?',
 '?',
 '4',
 '?',
 '?',
 '2',
 '2',
 '?',
 '1',
 '1',
 '1',
 '1',
 '1',
 '?',
 '1',
 '3',
 '2',
 '?',
 '?',
 '?',
 '?',
 '4',
 '?',
 '1',
 '1',
 '?',
 '?',
 '1',
 '?',
 '2',
 '1',
 '?',
 '?',
 '3',
 '?',
 '?',
 '2',
 '?',
 '?',
 '?',
 '1',
 '?',
 '?',
 '?',
 '?',
 '2',
 '4',
 '1',
 '?',
 '?',
 '?',
 '?',
 '1',
 '1',
 '1',
 '?',
 '?',
 '1',
 '1',
 '?',
 '2',
 '?',
 '?'

In [152]:
data = data.groupby(['sessionNo'], as_index=False).last() #granularização

In [153]:
diffCounts = (data['cCount']-data['bCount']).values

In [154]:
diffCounts # diferença entre cCount e bCount

array([ 0,  0,  9, ..., 74,  6,  0])

In [155]:
durCount = (data['duration']/data['cCount'])

In [156]:
durCount = durCount.replace(float("inf"), '?', regex=False).values

In [157]:
durCount #Razão entre a duração e o Ccount

array([39887.0, '?', 31055.727272727272, ..., 77777.68831168831,
       102191.57142857143, 82943.0], dtype=object)

In [158]:
data.groupby(['availability'], as_index=False).last()


,availability,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,...,onlineStatus,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
0,?,49998,18,7,5988882.0,77,9.99,149.99,5253.28,3,...,?,?,?,?,?,?,?,?,?,n
1,completely not determinable,49533,18,7,4305367.0,40,5.99,69.95,943.45,0,...,n,?,?,?,?,?,?,?,?,n
2,completely not orderable,49974,18,7,1832336.0,42,10.0,79.95,630.62,0,...,y,25023,2700,503,223,19,56,1,35,n
3,completely orderable,50000,18,7,82943.0,1,6.99,6.99,6.99,1,...,y,47,300,537,5,0,53,1,34,n
4,mainly not determinable,39468,10,7,874092.0,11,9.99,29.99,244.77,4,...,n,?,?,?,?,?,?,?,?,n
5,mainly not orderable,48329,17,7,1808881.0,106,1.0,24.99,411.92,3,...,y,24151,600,510,84,24,35,2,37,n
6,mainly orderable,49992,18,7,389549.0,16,5.99,9.99,127.84,4,...,y,?,?,?,?,?,?,?,?,n
7,mixed,49823,18,7,662571.0,7,1039.99,1299.99,7019.94,2,...,y,?,?,?,?,?,?,?,?,n


In [159]:

data.loc[(data.availability == 'completely not determinable'),'availability']= 0
data.loc[(data.availability == 'completely not orderable'),'availability']= 1
data.loc[(data.availability == 'completely orderable'),'availability']= 2
data.loc[(data.availability == 'mainly not determinable'),'availability']= 3
data.loc[(data.availability == 'mainly not orderable'),'availability']= 4
data.loc[(data.availability == 'mainly orderable'),'availability']= 5
data.loc[(data.availability == 'mixed'),'availability']= 6

In [160]:
data.loc[(data.onlineStatus == 'y'),'onlineStatus']= 1
data.loc[(data.onlineStatus == 'n'),'onlineStatus']= 0

In [161]:
data.loc[(data.order == 'y'),'order']= 1
data.loc[(data.order == 'n'),'order']= 0

In [162]:
data.insert(23, "bStep_count", bStep_count , True) 

In [163]:
data.insert(24, "countLog", countLog , True) 

In [164]:
data.insert(25, "modes", modes , True) 

In [165]:
data.insert(26, "diffCounts", diffCounts , True) 

In [166]:
data.insert(27, "durCount", durCount, True) 

In [167]:
data

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,payments,age,address,lastOrder,bStep_count,countLog,modes,diffCounts,durCount,order
0,1,6,5,39887.0,1,59.99,59.99,59.99,1,59.99,...,1,43,1,49,0,3,?,0,39887,1
1,2,6,5,94469.0,0,?,?,?,0,?,...,?,?,?,?,2,5,4,0,?,1
2,3,6,5,341613.0,11,9.99,29.99,109.95,2,9.99,...,12,45,1,11,1,5,?,9,31055.7,1
3,4,6,5,42812.0,4,4.99,4.99,19.96,1,4.99,...,?,?,?,?,0,2,1,3,10703,0
4,5,6,5,2816046.0,45,12.99,179.95,1093.72,4,19.99,...,1,46,1,40,2,8,?,41,62578.8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,18,7,7271812.0,197,6.99,59.99,4315.03,13,9.99,...,?,?,?,?,5,31,?,184,36912.8,1
49996,49997,18,7,667219.0,8,70.99,89.99,667.88,1,89.99,...,?,?,?,?,0,2,2,7,83402.4,0
49997,49998,18,7,5988882.0,77,9.99,149.99,5253.28,3,49.95,...,?,?,?,?,0,3,?,74,77777.7,0
49998,49999,18,7,715341.0,7,59.0,649.99,1159.95,1,89.99,...,0,24,1,4,0,2,1,6,102192,0
